# Projeto de Web Scraping

### Ele entra no site da investing.com na área de bancos centrais e pega os dados dos banco e salva um log em um arquivo.txt

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from os import getcwd
from datetime import datetime

In [2]:
def formatar_data(agora: datetime.now()):
    hora = agora.hour
    minuto = agora.minute
    segundo = agora.second
    if hora < 10:
        hora = f'0{hora}'
    if minuto < 10:
        minuto = f'0{minuto}'
    if segundo < 10:
        segundo = f'0{segundo}'
    return f'{agora.day}-{agora.month}-{agora.year} {hora};{minuto};{segundo}'

In [3]:
# opções do navegador
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # rodar em segundo plano
# abrir navegador
navegador = webdriver.Chrome(options=options)
# entrar no site
navegador.get('https://br.investing.com/central-banks/')

In [4]:
def formatar_taxa(banco):
    return float(banco['Taxa Atual'].replace(',', '.').replace('%', ''))

In [5]:
# pegar os dados
bancos_centrais = [
    # {'Banco Central': ..., 'Taxa Atual': x%, 'Reunião': xx.xx.xxxx, 'Última Mudança': xx.xx.xxxx (xxbp)}
]
for i in range(1, 12):
    banco_central = navegador.find_element(By.XPATH, f'//*[@id="curr_table"]/tbody/tr[{i}]/td[2]/a').text
    banco_central += ' ' + navegador.find_element(By.XPATH, f'//*[@id="curr_table"]/tbody/tr[{i}]/td[2]/span').text
    taxa_atual = navegador.find_element(By.XPATH, f'//*[@id="curr_table"]/tbody/tr[{i}]/td[3]').text
    reuniao = navegador.find_element(By.XPATH, f'//*[@id="curr_table"]/tbody/tr[{i}]/td[4]').text
    ultima_mudanca = navegador.find_element(By.XPATH, f'//*[@id="curr_table"]/tbody/tr[{i}]/td[5]').text
    bancos_centrais.append({
        'Banco Central': banco_central,
        'Taxa Atual': taxa_atual,
        'Reunião': reuniao,
        'Última Mudança': ultima_mudanca
    })

In [6]:
navegador.quit()

In [7]:
# formatar os dados
texto = f'{"Banco Central":<48}{"Taxa Atual":<20}{"Reunião":<20}{"Última Mudança"}\n\n'
for bc in bancos_centrais:
    texto += f'{bc["Banco Central"]:<48}{bc["Taxa Atual"]:<20}{bc["Reunião"]:<20}{bc["Última Mudança"]}\n'

In [8]:
# indicadores borocoxô
melhor_taxa_atual = max(bancos_centrais, key=lambda banco: formatar_taxa(banco))['Banco Central']
pior_taxa_atual = min(bancos_centrais, key=lambda banco: formatar_taxa(banco))['Banco Central']

In [9]:
# adicionar ao texto
texto += f'\nBanco com a melhor taxa atual: {melhor_taxa_atual}'
texto += f'\nBanco com a pior taxa atual: {pior_taxa_atual}'

In [10]:
print(texto)

Banco Central                                   Taxa Atual          Reunião             Última Mudança

Banco Central do Brasil (BCB)                   13,75%              07.12.2022          03.08.2022 (50bp)
Banco Central Europeu (ECB)                     2,00%               15.12.2022          27.10.2022 (75bp)
Reserva Federal (FED)                           3,25%               02.11.2022          21.09.2022 (75bp)
Banco da Inglaterra (BOE)                       2,25%               03.11.2022          22.09.2022 (50bp)
Banco Nacional Suíço (SNB)                      0,50%               15.12.2022          22.09.2022 (75bp)
Banco da Reserva da Austrália (RBA)             2,60%               06.12.2022          04.10.2022 (25bp)
Banco do Canadá (BOC)                           3,75%               07.12.2022          26.10.2022 (50bp)
Banco do Japão (BOJ)                            -0,10%              28.10.2022          29.01.2016 (-20bp)
Banco Central da Federação Russa (CBR)         

In [11]:
now = formatar_data(datetime.now())

In [12]:
# criar arquivo
arquivo = open(fr'logs/log {now}.txt', 'a')
# escrever no arquivo
arquivo.writelines(texto)
# fechar arquivo
arquivo.close()

In [13]:
print(fr'Arquivo salvo em: {getcwd()}\logs\log {now}')

Arquivo salvo em: C:\Users\Administrator\Documents\GitHub\Projetos\projeto-2\logs\log 28-10-2022 21;36;42
